In [1]:
import pandas as pd
from pandas import json_normalize
import requests
import json
import snowflake.connector

from dotenv import load_dotenv
import os
import http.client
import urllib.parse


from googletrans import Translator
import string
import re

from sqlalchemy import create_engine

In [2]:
load_dotenv()
rapidapi_key = os.getenv('RAPIDAPI_KEY')
rapidapi_host = "linkedin-job-search-api.p.rapidapi.com"
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')

In [10]:
def extract_linkedin_job_data():

    headers = {
        'x-rapidapi-key': rapidapi_key,
        'x-rapidapi-host': rapidapi_host
    }

    location = "Australia"
    limit = 100
    offset = 0
    titles = ["Data Engineer" , "Data Scientist", "Data Analyst"]

    df_daily_all = pd.DataFrame()

    for title_filter in titles:

        #URL code the title filter and location filter
        title_encoded=urllib.parse.quote(title_filter)
        location_encoded=urllib.parse.quote(location)

        #API endpoint 7day data query
        #base_url = f"/active-jb-7d?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"

        #API endpoint last 24 hour data query
        base_url = f"/active-jb-24h?limit={limit}&offset={offset}&title_filter={title_encoded}&location_filter={location_encoded}"
        url =  f"https://{rapidapi_host}{base_url}"


        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            # Convert the JSON data to a DataFrame
            df_daily = json_normalize(data)
            df_daily['job_category'] = title_filter
            print(title_filter, df_daily.shape)
            df_daily_all = pd.concat([df_daily_all, df_daily], ignore_index=True)


        else:
            print(f"Error: {response.status_code}")

    return df_daily_all

df_daily_all = extract_linkedin_job_data()
# df_daily_all

Data Engineer (9, 45)
Data Scientist (2, 45)
Data Analyst (10, 45)


In [11]:
df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.head()

,id,date_posted,date_created,title,organization,organization_url,date_validthrough,locations_raw,location_type,location_requirements_raw,...,linkedin_org_type,linkedin_org_foundeddate,linkedin_org_specialties,linkedin_org_locations,linkedin_org_description,linkedin_org_recruitment_agency_derived,seniority,directapply,linkedin_org_slug,job_category
0,1624159820,2025-05-05T06:03:59,2025-05-05T06:22:13.284745,Data Engineer Manager (SaaS),CreditorWatch,https://www.linkedin.com/company/creditor-watch,2025-06-04T06:03:25,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,NaN,...,Privately Held,2010.0,"['Commercial Credit Risk', 'Commercial Monitor...","['Sydney, NSW 2000, AU']",CreditorWatch is a leading Australian data and...,False,Intermedio,True,creditor-watch,Data Engineer
1,1624047094,2025-05-05T05:21:01,2025-05-05T05:35:13.399156,Full Stack Developer/ Senior Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-04T05:21:01,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,NaN,...,Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank,Data Engineer
2,1623877317,2025-05-05T02:55:02,2025-05-05T04:22:08.424003,Senior Data Engineer,Swyftx,https://www.linkedin.com/company/swyftx,2025-06-04T02:55:00,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,NaN,...,Privately Held,2018.0,[''],"['Milton, QLD 4064, AU']","Founded in 2018, Swyftx is one of Australia an...",False,Mid-Senior level,False,swyftx,Data Engineer
3,1623290174,2025-05-05T00:23:27,2025-05-05T00:32:35.862355,Azure Data Engineer,HCLTech,https://www.linkedin.com/company/hcltech,2025-06-04T00:23:27,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,NaN,...,Public Company,NaN,"['Manufacturing', 'Aerospace & Defense', 'Fina...","['Noida, Uttar Pradesh 201301, IN', '330 Potre...","HCLTech is a global technology company, home t...",False,Mid-Senior level,True,hcltech,Data Engineer
4,1623164199,2025-05-04T23:25:12,2025-05-04T23:33:33.082861,Senior Data Engineer,easyA - Recruitment made easy,https://www.linkedin.com/company/easy-authoring,2025-06-03T23:25:12,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,NaN,...,Privately Held,2006.0,"['Recruitment Services', 'Permanent Recruitmen...","['Level 35, Tower One, International Towers, 1...",easyA is a leading recruitment agency speciali...,True,Mid-Senior level,True,easy-authoring,Data Engineer


In [12]:
def get_clean_data_jobs(df_daily_all):
     pattern=re.compile(r'\bData Engineer\b|\bData Scientist\b|\bData Analyst\b' , re.IGNORECASE)
     df_daily_all = df_daily_all[df_daily_all['title'].str.contains(pattern)]
     return df_daily_all

df_daily_all = get_clean_data_jobs(df_daily_all)
df_daily_all.shape

(15, 45)

In [13]:
df_daily_all.to_csv('linkedin_jobs_daily.csv', index=False)

In [14]:
df_daily_all = pd.read_csv('linkedin_jobs_daily.csv')
df_daily_all.shape

(15, 45)

In [15]:
def update_columns(df_daily_all):
    df_daily_all.columns = df_daily_all.columns.str.upper()
    df_daily_all = df_daily_all[['ID', 'DATE_POSTED', 'DATE_CREATED', 'TITLE', 'JOB_CATEGORY',
       'ORGANIZATION', 'ORGANIZATION_URL', 'DATE_VALIDTHROUGH', 'LOCATIONS_RAW',
       'LOCATION_TYPE', 'LOCATION_REQUIREMENTS_RAW', 'EMPLOYMENT_TYPE', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN', 'ORGANIZATION_LOGO',
       'CITIES_DERIVED', 'REGIONS_DERIVED', 'COUNTRIES_DERIVED',
       'LOCATIONS_DERIVED', 'TIMEZONES_DERIVED', 'LATS_DERIVED',
       'LNGS_DERIVED', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_EMPLOYEES', 'LINKEDIN_ORG_URL',
       'LINKEDIN_ORG_SIZE', 'LINKEDIN_ORG_SLOGAN', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_FOLLOWERS', 'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION', 'LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY', 'LINKEDIN_ORG_SLUG']]
    return df_daily_all

df_daily_all = update_columns(df_daily_all)
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1624159820,2025-05-05T06:03:59,2025-05-05T06:22:13.284745,Data Engineer Manager (SaaS),Data Engineer,CreditorWatch,https://www.linkedin.com/company/creditor-watch,2025-06-04T06:03:25,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Privately Held,2010.0,"['Commercial Credit Risk', 'Commercial Monitor...","['Sydney, NSW 2000, AU']",CreditorWatch is a leading Australian data and...,False,Intermedio,True,creditor-watch
1,1624047094,2025-05-05T05:21:01,2025-05-05T05:35:13.399156,Full Stack Developer/ Senior Data Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-04T05:21:01,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Sydney, NSW",Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank


### Process the df_daily_all data 


In [16]:
def extract_job_date(date_created):
    #extract the job_date from job_created
    df_daily_all['job_date'] = pd.to_datetime(df_daily_all['DATE_CREATED']).dt.date
    return df_daily_all['job_date']

#Apply the function to the DataFrame
df_daily_all['job_date'] = extract_job_date(df_daily_all['DATE_CREATED'])
df_daily_all['job_date'][:5]


0    2025-05-05
1    2025-05-05
2    2025-05-05
3    2025-05-05
4    2025-05-04
Name: job_date, dtype: object

In [17]:
#Extrac the job city and state from the LOCATIONS_RAW field

def extract_city(list):
    """
    Extracts the city from the given text using regex.
    """
    # Regex pattern to match the city in the LOCATIONS_RAW field
    text = str(list)
    city_pattern = r"'addressLocality':\s*'(.*)',\s'addressRegion':"
    match = re.search(city_pattern, text)
    if match:
        city = match.group(1)
        if 'sidney' in city.lower() or 'sídney' in city.lower() or '悉尼' in city.lower(): #fix wrong city name
            return "Sydney"
        return city
    else:
        return None



def extract_state(list):
    """
    Extracts the state from the given text using regex.
    """
    # Regex pattern to match the region/state in the LOCATIONS_RAW field
    text = str(list)
    state_pattern = r"'addressRegion':\s*(.*)',\s'streetAddress'"
    match = re.search(state_pattern, text)
    if match:
        state = match.group(1)
        state= state.replace("'", "").strip()
        return state
    else:
        return None


#Extract city and state from Locations Raw
df_daily_all['city'] = df_daily_all['LOCATIONS_RAW'].apply(extract_city)


df_daily_all['state'] = df_daily_all['LOCATIONS_RAW'].apply(extract_state)



df_daily_all.head()

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,job_date,city,state
0,1624159820,2025-05-05T06:03:59,2025-05-05T06:22:13.284745,Data Engineer Manager (SaaS),Data Engineer,CreditorWatch,https://www.linkedin.com/company/creditor-watch,2025-06-04T06:03:25,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Commercial Credit Risk', 'Commercial Monitor...","['Sydney, NSW 2000, AU']",CreditorWatch is a leading Australian data and...,False,Intermedio,True,creditor-watch,2025-05-05,Sydney,NSW
1,1624047094,2025-05-05T05:21:01,2025-05-05T05:35:13.399156,Full Stack Developer/ Senior Data Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-04T05:21:01,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank,2025-05-05,Woden,None
2,1623877317,2025-05-05T02:55:02,2025-05-05T04:22:08.424003,Senior Data Engineer,Data Engineer,Swyftx,https://www.linkedin.com/company/swyftx,2025-06-04T02:55:00,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,[''],"['Milton, QLD 4064, AU']","Founded in 2018, Swyftx is one of Australia an...",False,Mid-Senior level,False,swyftx,2025-05-05,Brisbane,QLD
3,1623290174,2025-05-05T00:23:27,2025-05-05T00:32:35.862355,Azure Data Engineer,Data Engineer,HCLTech,https://www.linkedin.com/company/hcltech,2025-06-04T00:23:27,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Manufacturing', 'Aerospace & Defense', 'Fina...","['Noida, Uttar Pradesh 201301, IN', '330 Potre...","HCLTech is a global technology company, home t...",False,Mid-Senior level,True,hcltech,2025-05-05,Sydney,NSW
4,1623164199,2025-05-04T23:25:12,2025-05-04T23:33:33.082861,Senior Data Engineer,Data Engineer,easyA - Recruitment made easy,https://www.linkedin.com/company/easy-authoring,2025-06-03T23:25:12,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"['Recruitment Services', 'Permanent Recruitmen...","['Level 35, Tower One, International Towers, 1...",easyA is a leading recruitment agency speciali...,True,Mid-Senior level,True,easy-authoring,2025-05-04,Brisbane,QLD


In [18]:
#Extract the employment type from the EMPLOYMENT_TYPE field

def extract_employment_type(df_daily_all):
    """
    Extracts the employment type from the given text using regex.
    """
    df_daily_all['employment_type'] = (
        df_daily_all['EMPLOYMENT_TYPE']
        .astype(str)
        .str.replace(r"[\[\]']", '', regex=True)
        .str.strip()
    )

    df_daily_all.drop(columns=['EMPLOYMENT_TYPE'], inplace=True)

    return df_daily_all['employment_type']


df_daily_all['employment_type'] = extract_employment_type(df_daily_all)
df_daily_all['employment_type'].tail()

10    CONTRACTOR
11     FULL_TIME
12     FULL_TIME
13    CONTRACTOR
14     FULL_TIME
Name: employment_type, dtype: object

In [19]:
#Extract the employment size
def extract_employee_size(LINKEDIN_ORG_SIZE):
    """
    Extracts the employee size from the given text using regex.
    """
    df_daily_all['org_size'] = (
        df_daily_all['LINKEDIN_ORG_SIZE']
        .astype(str)
        .str.replace(r"employees", '', regex=True)
        .str.strip()
    )

    return df_daily_all['org_size']


df_daily_all['org_size'] = extract_employee_size(df_daily_all['LINKEDIN_ORG_SIZE'])
df_daily_all['org_size'].tail()

10         10,001+
11         10,001+
12           11-50
13             nan
14    5,001-10,000
Name: org_size, dtype: object

In [20]:
df_daily_all.columns = df_daily_all.columns.str.upper()
df_daily_all.head(2)

,ID,DATE_POSTED,DATE_CREATED,TITLE,JOB_CATEGORY,ORGANIZATION,ORGANIZATION_URL,DATE_VALIDTHROUGH,LOCATIONS_RAW,LOCATION_TYPE,...,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORG_SIZE
0,1624159820,2025-05-05T06:03:59,2025-05-05T06:22:13.284745,Data Engineer Manager (SaaS),Data Engineer,CreditorWatch,https://www.linkedin.com/company/creditor-watch,2025-06-04T06:03:25,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,CreditorWatch is a leading Australian data and...,False,Intermedio,True,creditor-watch,2025-05-05,Sydney,NSW,FULL_TIME,201-500
1,1624047094,2025-05-05T05:21:01,2025-05-05T05:35:13.399156,Full Stack Developer/ Senior Data Engineer,Data Engineer,Peoplebank,https://www.linkedin.com/company/peoplebank,2025-06-04T05:21:01,"[{'@type': 'Place', 'address': {'@type': 'Post...",NaN,...,"Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank,2025-05-05,Woden,None,CONTRACTOR,201-500


In [21]:
#Only keep the relevant columns

df_daily_all = df_daily_all[['ID', 'TITLE', 'JOB_CATEGORY',
       'JOB_DATE', 'CITY', 'STATE', 'EMPLOYMENT_TYPE' ,
       'ORGANIZATION', 'ORGANIZATION_URL', 'URL',
       'SOURCE_TYPE', 'SOURCE', 'SOURCE_DOMAIN',
       'ORGANIZATION_LOGO', 'REMOTE_DERIVED', 'RECRUITER_NAME', 'RECRUITER_TITLE',
       'RECRUITER_URL', 'LINKEDIN_ORG_URL',
       'ORG_SIZE', 'LINKEDIN_ORG_INDUSTRY',
       'LINKEDIN_ORG_HEADQUARTERS',
       'LINKEDIN_ORG_TYPE', 'LINKEDIN_ORG_FOUNDEDDATE',
       'LINKEDIN_ORG_SPECIALTIES', 'LINKEDIN_ORG_LOCATIONS',
       'LINKEDIN_ORG_DESCRIPTION','LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED',
       'SENIORITY', 'DIRECTAPPLY',
       'LINKEDIN_ORG_SLUG']]


df_daily_all.head()

,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1624159820,Data Engineer Manager (SaaS),Data Engineer,2025-05-05,Sydney,NSW,FULL_TIME,CreditorWatch,https://www.linkedin.com/company/creditor-watch,https://au.linkedin.com/jobs/view/data-enginee...,...,"Sydney, NSW",Privately Held,2010.0,"['Commercial Credit Risk', 'Commercial Monitor...","['Sydney, NSW 2000, AU']",CreditorWatch is a leading Australian data and...,False,Intermedio,True,creditor-watch
1,1624047094,Full Stack Developer/ Senior Data Engineer,Data Engineer,2025-05-05,Woden,None,CONTRACTOR,Peoplebank,https://www.linkedin.com/company/peoplebank,https://au.linkedin.com/jobs/view/full-stack-d...,...,"Sydney, NSW",Privately Held,1990.0,"['Digital / Digital Transformation', 'Business...","['345 George Street, Level 13, Sydney, NSW 200...","Peoplebank, part of RGF Staffing ANZ is a lead...",True,Mid-Senior level,False,peoplebank
2,1623877317,Senior Data Engineer,Data Engineer,2025-05-05,Brisbane,QLD,FULL_TIME,Swyftx,https://www.linkedin.com/company/swyftx,https://au.linkedin.com/jobs/view/senior-data-...,...,"Milton, QLD",Privately Held,2018.0,[''],"['Milton, QLD 4064, AU']","Founded in 2018, Swyftx is one of Australia an...",False,Mid-Senior level,False,swyftx
3,1623290174,Azure Data Engineer,Data Engineer,2025-05-05,Sydney,NSW,FULL_TIME,HCLTech,https://www.linkedin.com/company/hcltech,https://au.linkedin.com/jobs/view/azure-data-e...,...,"Noida, Uttar Pradesh",Public Company,NaN,"['Manufacturing', 'Aerospace & Defense', 'Fina...","['Noida, Uttar Pradesh 201301, IN', '330 Potre...","HCLTech is a global technology company, home t...",False,Mid-Senior level,True,hcltech
4,1623164199,Senior Data Engineer,Data Engineer,2025-05-04,Brisbane,QLD,FULL_TIME,easyA - Recruitment made easy,https://www.linkedin.com/company/easy-authoring,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, NSW",Privately Held,2006.0,"['Recruitment Services', 'Permanent Recruitmen...","['Level 35, Tower One, International Towers, 1...",easyA is a leading recruitment agency speciali...,True,Mid-Senior level,True,easy-authoring


## Connect to  Snowflake database for raw data initial processing

In [22]:
#Establish a connection to Snowflake

def connect_to_snowflake():
    try:

        conn = snowflake.connector.connect(
            user="NIKKILW2025",
            password=snowflake_password,
            account="gbszkwp-by30611",
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
        print("Connection to Snowflake established successfully.")
        return conn
    except Exception as e:
        print(f"Error connecting to Snowflake: {e}")
        return None

conn = connect_to_snowflake()

Connection to Snowflake established successfully.


In [23]:
#query the raw data and narrow down to DE, DS and DA roles
#This data is filtered by relevant roles, and translated into English (but no city, state and seniority fix)
def query_existing_job_data(conn):
    query = """
        SELECT * FROM LINKEDIN_JOB_API_CLEANED_DATA
        WHERE (
            lower(TITLE) LIKE '%data engineer%'
            OR lower(TITLE) LIKE '%data scientist%'
            OR lower(TITLE) LIKE '%data analyst%'
            )
    """

    df = pd.read_sql(query, conn)
    print(df.shape)
    return df

df = query_existing_job_data(conn)
df.head()

/tmp/ipykernel_72056/824833193.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


(276, 31)


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
0,1599037955,Senior Data Engineer,Data Engineer,2025-04-28,Greater Sydney Area,NA,FULL_TIME,One51 | Data & Analytics Consultancy,https://www.linkedin.com/company/one51consulting,https://au.linkedin.com/jobs/view/senior-data-...,...,"Sydney, NSW",Privately Held,2020.0,"['Business Intelligence', 'Data Warehousing', ...","['333 George Street, Level 13, Sydney, NSW 200...",Drawing on a wealth of expertise and a deep un...,False,Mid-Senior level,False,one51consulting
1,1598878880,Senior Data Engineer,Data Engineer,2025-04-28,Brisbane,QLD,CONTRACTOR,Data#3,https://www.linkedin.com/company/data3,https://au.linkedin.com/jobs/view/senior-data-...,...,"Toowong, Queensland",Public Company,1977.0,"['Cloud Solutions', 'Mobility Solutions', 'Sec...","['555 Coronation Dr, Toowong, Queensland 4066,...","Data#3 Limited (DTL), is focused on helping cu...",False,Medium-high level,False,data3
2,1598880069,Data Engineer,Data Engineer,2025-04-28,Sydney,NSW,CONTRACTOR,Whizdom,https://www.linkedin.com/company/whizdom-recru...,https://au.linkedin.com/jobs/view/data-enginee...,...,"Canberra, Australian Capital Territory",Privately Held,2006.0,"['IT Recruitment', 'Recruitment for Government...","['28-34 Thynne St, Unit 7, Canberra, Australia...","Established in 2006, Whizdom is an Australian ...",True,Middle level,True,whizdom-recruitment
3,1598878894,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Moreton Bay,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None
4,1598880083,Senior Data Engineer and Business Intelligence...,Data Engineer,2025-04-28,Sunshine Coast,QLD,FULL_TIME,University of the Sunshine Coast,https://www.linkedin.com/school/university-of-...,https://au.linkedin.com/jobs/view/senior-data-...,...,None,None,NaN,None,None,None,None,Not Applicable,False,None


In [24]:
df.to_csv('api_cleaned_job_id_snowflake.csv', index=False)

In [25]:
list_daily = df_daily_all.columns.tolist()
list_df = df.columns.tolist()
list_daily ==  list_df

True

In [26]:
#Check the Job ID from df and only keep those new jobs based on the Job IDs
def keep_new_jobs():
    existing_job_ids = df.ID.unique().tolist()

    df_new_jobs = df_daily_all[~(df_daily_all['ID'].isin(existing_job_ids))].reset_index(drop=True)
    print(f'{df_new_jobs.shape[0]} new jobs ready to load to Snowflake')
    return df_new_jobs

df_new_jobs = keep_new_jobs()
df_new_jobs.tail()

15 new jobs ready to load to Snowflake


,ID,TITLE,JOB_CATEGORY,JOB_DATE,CITY,STATE,EMPLOYMENT_TYPE,ORGANIZATION,ORGANIZATION_URL,URL,...,LINKEDIN_ORG_HEADQUARTERS,LINKEDIN_ORG_TYPE,LINKEDIN_ORG_FOUNDEDDATE,LINKEDIN_ORG_SPECIALTIES,LINKEDIN_ORG_LOCATIONS,LINKEDIN_ORG_DESCRIPTION,LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED,SENIORITY,DIRECTAPPLY,LINKEDIN_ORG_SLUG
10,1624159796,Oracle Data Analyst - CC&B,Data Analyst,2025-05-05,墨尔本,VIC,CONTRACTOR,DXC Technology,https://www.linkedin.com/company/dxctechnology,https://au.linkedin.com/jobs/view/oracle-data-...,...,"Ashburn, Virginia",Public Company,2017.0,"['Insurance BPaaS and BPO', 'Analytics and Eng...","['20408 Bashan Dr, 231, Ashburn, Virginia 2014...",DXC Technology (NYSE: DXC) helps global compan...,False,中高级,False,dxctechnology
11,1623593398,Senior Technical Data Analyst,Data Analyst,2025-05-05,Sydney,NSW,FULL_TIME,Commonwealth Bank,https://www.linkedin.com/company/commonwealthbank,https://au.linkedin.com/jobs/view/senior-techn...,...,"Sydney, NSW",Public Company,NaN,[''],"['., Sydney, NSW 2000, AU']",Australia’s leading provider of financial serv...,False,Not Applicable,False,commonwealthbank
12,1623345706,Operations & Data Analyst,Data Analyst,2025-05-05,Sydney,NSW,FULL_TIME,BON CHARGE,https://www.linkedin.com/company/boncharge,https://au.linkedin.com/jobs/view/operations-d...,...,"Perth, Western Australia",Privately Held,2017.0,"['optics', 'artificial light', 'bluelight', 'p...","['PO Box 7574, Cloisters Square PO, Perth, Wes...","At BON CHARGE, we are dedicated to 'simplifyin...",False,Entry level,True,boncharge
13,1623250291,Data Analyst,Data Analyst,2025-05-05,North Ryde,NSW,CONTRACTOR,Macquarie University,https://www.linkedin.com/school/macquarie-univ...,https://au.linkedin.com/jobs/view/data-analyst...,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Not Applicable,False,macquarie-university
14,1621226288,"Data Analyst (Bangkok Based, relocation provided)",Data Analyst,2025-05-04,Melbourne,VIC,FULL_TIME,Agoda,https://www.linkedin.com/company/agoda,https://au.linkedin.com/jobs/view/data-analyst...,...,"Singapore, Singapore",Public Company,2005.0,"['internet', 'software engineering', 'online t...","['20 Cecil Street, #1401, GSH Plaza, Singapore...","At Agoda, we bridge the world through travel. ...",False,Associate,False,agoda


In [27]:
#Translate new job's business name, city, state and seniority to English

translator = Translator()

def translate_text(text, target_language='en'):
    try:
        if not text or pd.isna(text):
            return 'NA'
        translated_text = translator.translate(str(text), dest=target_language)
        return translated_text.text
    except Exception as e:
        print(f"Error translating text: {e} (Text: {text})")
        return text

for col in ['CITY', 'STATE', 'ORGANIZATION', 'SENIORITY']:
    unique_values = df_new_jobs[col].dropna().unique()
    translation_map = {val: translate_text(val, target_language='en') for val in unique_values}
    df_new_jobs[col] = df_new_jobs[col].map(translation_map).fillna('NA')

In [28]:
df_new_jobs['CITY'].unique()

array(['Sydney', 'Woden', 'Brisbane', 'KULUIN', 'Melburn', 'Melbourne',
       'North Ryde'], dtype=object)

In [29]:
df_new_jobs['STATE'].unique()

array(['NSW', 'NA', 'QLD', 'VIC'], dtype=object)

In [30]:

def load_to_snowflake(df_new_jobs):
    # Create a Snowflake connection engine
    engine = create_engine(
        'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'.format(
            user="NIKKILW2025",
            password=snowflake_password,
            account="gbszkwp-by30611",
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
    )

    table_name = "linkedin_job_api_cleaned_data"

    df_new_jobs.to_sql(
        name=table_name,
        con=engine,
        if_exists='append', #append data
        index=False
    )

    print(f"Data loaded to Snowflake table {table_name} successfully.")


load_to_snowflake(df_new_jobs)

Data loaded to Snowflake table linkedin_job_api_cleaned_data successfully.
